<a href="https://colab.research.google.com/github/lazybuttrying/AI_Project/blob/main/leaf_segmentation/MaskRCNN/maskImage_to_annotationJSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


### 모든 사진의 이름이 겹치지 않도록 조정

In [5]:
cd /gdrive/MyDrive/AI/maskrcnn/training/

/gdrive/MyDrive/AI/maskrcnn


In [76]:
import os

len(os.listdir("./A1"))//4, len(os.listdir("./A2"))//4, len(os.listdir("./A3"))//4, len(os.listdir("./A4"))//4,

(128, 31, 27, 624)

In [75]:
import os

for folder in ['A1', 'A2', 'A3', 'A4']:
  for fname in os.listdir("./"+folder):
    half = fname.find("_")
    old = ("./"+folder+"/"+fname)
    new = ("./"+folder+"/"+fname[:half]+ "~" + folder + fname[half:])
    os.rename(old, new)

# Image to COCO annotation

In [ ]:
cd /gdrive/MyDrive/AI/maskrcnn/

In [ ]:
!git clone https://github.com/chrise96/image-to-coco-json-converter.git

Cloning into 'image-to-coco-json-converter'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 58 (delta 12), reused 51 (delta 10), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [13]:
cd image-to-coco-json-converter

/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter


In [14]:
!pip install -r requirements.txt

In [54]:
cd dataset

/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset


### label 이미지 마련하기

In [55]:
!mkdir leaf_img
!mkdir leaf_img/train_mask
!mkdir leaf_img/val_mask

In [ ]:
cd /gdrive/MyDrive/AI/maskrcnn/training/

In [77]:
# train 752, val 58

!cp A1/*_label.png /gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/leaf_img/train_mask
!cp A2/*_label.png /gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/leaf_img/val_mask
!cp A3/*_label.png /gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/leaf_img/val_mask
!cp A4/*_label.png /gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/leaf_img/train_mask

In [79]:
cd /gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/leaf_img

/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/leaf_img


In [82]:
# label 이름으로 저장되면 나중에 사진 이름과 매치되지 않아서 작동 안 됨
# json에 기록될 파일 이름이 실제 파일과 같아지도록 수정

import os

for path in ['./train_mask', './val_mask']:
  for fname in os.listdir(path):
    # print(fname[:9]+"rgb.png")
    half = fname.find("_")
    old = path+"/"+fname
    new = path+"/"+fname[:half]+"_rgb.png"
    os.rename(old, new)

### 변환 코드

In [83]:
import glob

from src.create_annotations import *
def get_json(mask_image, width, height):
    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    cnt = 1
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))[:3]
        
            if pixel != (0, 0, 0): # 배경은 제외
              # Check to see if we have created a sub-mask...
              pixel_str = str(pixel)
              sub_mask = sub_masks.get(pixel_str)
              if sub_mask is None:
                # Create a sub-mask (one bit per pixel) and add to the dictionary
                  # Note: we add 1 pixel of padding in each direction
                  # because the contours module doesn"t handle cases
                  # where pixels bleed to the edge of the image
                  sub_masks[pixel_str] = cnt
                  cnt+=1

              # Set the pixel value to 1 (default is 0), accounting for padding
              #sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks

In [84]:
def create_sub_masks(mask_image, width, height):
    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))[:3]

            if pixel != (0, 0, 0):
              # Check to see if we have created a sub-mask...
              pixel_str = str(pixel)
              sub_mask = sub_masks.get(pixel_str)
              if sub_mask is None:
                # Create a sub-mask (one bit per pixel) and add to the dictionary
                  # Note: we add 1 pixel of padding in each direction
                  # because the contours module doesn"t handle cases
                  # where pixels bleed to the edge of the image
                  sub_masks[pixel_str] = Image.new("1", (width+2, height+2))

              # Set the pixel value to 1 (default is 0), accounting for padding
              sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks


In [85]:


# # Label ids of the dataset
# category_ids = {
#     "outlier": 0,
#     "window": 1,
#     "wall": 2,
#     "balcony": 3,
#     "door": 4,
#     "roof": 5,
#     "sky": 6,
#     "shop": 7,
#     "chimney": 8
# }

# # Define which colors match which categories in the images
# category_colors = {
#     "(0, 0, 0)": 0, # Outlier
#     "(255, 0, 0)": 1, # Window
#     "(255, 255, 0)": 2, # Wall
#     "(128, 0, 255)": 3, # Balcony
#     "(255, 128, 0)": 4, # Door
#     "(0, 0, 255)": 5, # Roof
#     "(128, 255, 255)": 6, # Sky
#     "(0, 255, 0)": 7, # Shop
#     "(128, 128, 128)": 8 # Chimney
# }

# # Define the ids that are a multiplolygon. In our case: wall, roof and sky
# multipolygon_ids = [2, 5, 6]



In [87]:
import gc
# Get "images" and "annotations" info 
def images_annotations_info(maskpath):
    # This id will be automatically increased as we go
    annotation_id = 0
    image_id = 0
    annotations = []
    images = []
    
    for mask_image in glob.glob(maskpath + "*.png"):
        # The mask image is *.png but the original image is *.jpg.
        # We make a reference to the original file in the COCO JSON file
        original_file_name = os.path.basename(mask_image).split(".")[0] + ".png"

        # Open the image and (to be sure) we convert it to RGB
        mask_image_open = Image.open(mask_image).convert("RGB")
        w, h = mask_image_open.size
        
        # "images" info 
        image = create_image_annotation(original_file_name, w, h, image_id)
        images.append(image)
        del image

        ###########category_colors = get_json(mask_image_open, w, h)
        # print(category_colors)

        sub_masks = create_sub_masks(mask_image_open, w, h)

        del mask_image_open
        gc.collect()

        for color, sub_mask in sub_masks.items():
            # print(sub_masks,category_colors.keys(), color)
          
            ###########category_id = category_colors[color]

            # "annotations" info
            polygons, segmentations = create_sub_mask_annotation(sub_mask)

            # # Check if we have classes that are a multipolygon
            # if category_id in multipolygon_ids:
            #     # Combine the polygons to calculate the bounding box and area
            #     multi_poly = MultiPolygon(polygons)
                                
            #     annotation = create_annotation_format(multi_poly, segmentations, image_id, category_id, annotation_id)

            #     annotations.append(annotation)
            #     annotation_id += 1
            # else:
            
            for i in range(len(polygons)):
                    # Cleaner to recalculate this variable
                    segmentation = [np.array(polygons[i].exterior.coords).ravel().tolist()]
                    
                    annotation = create_annotation_format(polygons[i], segmentation, image_id, i, annotation_id)
                    
                    annotations.append(annotation)
                    annotation_id += 1
        image_id += 1
    return images, annotations, annotation_id


In [31]:
get_coco_json_format()

{'annotations': [{}],
 'categories': [{}],
 'images': [{}],
 'info': {},
 'licenses': []}

In [90]:
cd /gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/

/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter


In [91]:
pwd

'/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter'

In [92]:
if __name__ == "__main__":
    # Get the standard COCO JSON format
    coco_format = get_coco_json_format()
    
    for keyword in ["train", "val"]:
        mask_path = "dataset/leaf_img/{}_mask/".format(keyword)
        
        # Create category section
        # coco_format["categories"] = create_category_annotation(category_ids)
    
        # Create images and annotations sections
        coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)
        with open("output/{}.json".format(keyword),"w") as outfile:
            json.dump(coco_format, outfile)
        
        print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))

Created 11517 annotations for images in folder: dataset/leaf_img/train_mask/
Created 463 annotations for images in folder: dataset/leaf_img/val_mask/


# COCO to VGG
https://annotate.officialstatistics.org/

coco 로 붙러와서 json으로 저장

via_region_data.json라는 이름으로 

google drive의 dataset/leaf_img/train_mask/ 와 dataset/leaf_img/val_mask/

각각에 올려두기. 



### VGG annotation 손보기

In [35]:
# import json
# import pprint


# with open('/content/via_region_data (3).json', 'r') as f:

#     json_data = json.load(f)

# # print(json.dumps(json_data, indent="\t") )

# for k,v in json_data.items():
#   tmp = {}
#   for i in range(len(v["regions"])):
#     #print(v["regions"])
#     v["regions"][i]["region_attributes"] = {}
#     tmp[i] = v["regions"][i]
#   #pprint.pprint(tmp)
#   v["regions"] = tmp

# with open('/content/via_region_data.json', 'w', encoding='utf-8') as make_file:

#     json.dump(json_data, make_file, indent="\t")

FileNotFoundError: ignored

# 마무리된 dataset 옮기기

In [102]:
pwd # 현재위치확인

'/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/leaf_img'

In [94]:
cd /gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/leaf_img/

/gdrive/MyDrive/AI/maskrcnn/image-to-coco-json-converter/dataset/leaf_img


In [100]:
!rm -d -r /gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/leaf_dataset/train
!rm -d -r /gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/leaf_dataset/val
!mkdir /gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/leaf_dataset/train
!mkdir /gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/leaf_dataset/val

In [101]:
!cp train_mask/* /gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/leaf_dataset/train/
!cp val_mask/* /gdrive/MyDrive/AI/maskrcnn/Mask_RCNN/leaf_dataset/val/